# Classification Example of BackPropagation

### Importing Libraries

In [9]:
import numpy as np
import pandas as pd

### Creating a Data Frame

In [10]:
df = pd.DataFrame([[8,8,1],[7,9,1],[6,10,0],[5,5,0]], columns=['cgpa', 'profile_score', 'placed'])

In [11]:
df.head()

,cgpa,profile_score,placed
0,8,8,1
1,7,9,1
2,6,10,0
3,5,5,0


In [12]:
def initialize_parameters(layer_dims):
    """
    This Function Initialize the value of parametrs like weights & Parameters 
    Initializes the parameters (weights and biases) for a neural network.
    
    Arguments:
    layer_dims -- List containing the dimensions of each layer in the network.
                  Example: [input_size, hidden_layer1_size, ..., output_size]

    Returns:
    parameters -- Python dictionary containing the initialized parameters:
                  parameters["W1"] = weight matrix for layer 1
                  parameters["b1"] = bias vector for layer 1
                  ...
                  parameters["WL"] = weight matrix for the last layer
                  parameters["bL"] = bias vector for the last layer
    """

    np.random.seed(3)  # Set a seed for the random number generator for reproducibility
    parameters = {}    # Initialize an empty dictionary to store the parameters
    L = len(layer_dims)  # Number of layers in the network, including input and output layers

    for l in range(1, L):
        # Initialize weight matrix W for layer l with small values (here, all ones multiplied by 0.1)
        parameters['W' + str(l)] = np.ones((layer_dims[l-1], layer_dims[l-1])) * 0.1
        
        # Initialize bias vector b for layer l with zeros
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

    return parameters  # Return the dictionary containing all the parameters


### Utility Functions
In classification Activation function in SIgmoid

In [13]:
def sigmoid(Z):
    # Compute the sigmoid activation function
    # Sigmoid function is defined as 1 / (1 + exp(-Z))
    # It maps the input 'Z' to a value between 0 and 1
    
    A = 1 / (1 + np.exp(-Z))  # Apply the sigmoid function to 'Z'
    
    return A  # Return the output 'A', which is the result of the sigmoid function


In [14]:
def linear_forward(A_prev, W, b):
    # Perform the linear part of a layer's forward propagation.
    # Compute the linear transformation: Z = W.T * A_prev + b
    
    Z = np.dot(W.T, A_prev) + b  # Calculate the linear combination of inputs and weights
    
    A = sigmoid(Z)  # Apply the sigmoid activation function to the linear output Z
    
    return A  # Return the activated output 'A'


In [15]:
def L_layer_forward(X, parameters):
    """
    Implements forward propagation for the entire neural network.

    Arguments:
    X -- Input data, of shape (input size, number of examples)
    parameters -- Python dictionary containing the parameters "W1", "b1", ..., "WL", "bL":
                  Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                  bl -- bias vector of shape (layer_dims[l], 1)

    Returns:
    A -- The output of the final layer, which is the prediction of the network
    A_prev -- The output of the second last layer (useful for certain cases like computing the gradient in backpropagation)
    """

    A = X  # Initialize activation for the input layer (A0 = X)
    L = len(parameters) // 2  # Number of layers in the neural network (dividing by 2 since each layer has W and b)

    # Loop over each layer to perform forward propagation
    for l in range(1, L+1):
        A_prev = A  # Store the activation from the previous layer
        
        # Retrieve the parameters for the current layer l
        Wl = parameters['W' + str(l)]
        bl = parameters['b' + str(l)]
        
        # Perform the linear step for the current layer
        A = linear_forward(A_prev, Wl, bl)
        
        # Uncomment these lines if you want to debug and see the values at each layer
        #print("A"+str(l-1)+": ", A_prev)
        #print("W"+str(l)+": ", Wl)
        #print("b"+str(l)+": ", bl)
        #print("--" * 20)
        #print("A"+str(l)+": ", A)
        #print("**" * 20)

    return A, A_prev  # Return the output of the final layer and the second last layer's activation


In [16]:
def update_parameters(parameters,y,y_hat,A1,X):
  parameters['W2'][0][0] = parameters['W2'][0][0] + (0.0001 * (y - y_hat)*A1[0][0])
  parameters['W2'][1][0] = parameters['W2'][1][0] + (0.0001 * (y - y_hat)*A1[1][0])
  parameters['b2'][0][0] = parameters['W2'][1][0] + (0.0001 * (y - y_hat))

  parameters['W1'][0][0] = parameters['W1'][0][0] + (0.0001 * (y - y_hat)*parameters['W2'][0][0]*A1[0][0]*(1-A1[0][0])*X[0][0])
  parameters['W1'][0][1] = parameters['W1'][0][1] + (0.0001 * (y - y_hat)*parameters['W2'][0][0]*A1[0][0]*(1-A1[0][0])*X[1][0])
  parameters['b1'][0][0] = parameters['b1'][0][0] + (0.0001 * (y - y_hat)*parameters['W2'][0][0]*A1[0][0]*(1-A1[0][0]))

  parameters['W1'][1][0] = parameters['W1'][1][0] + (0.0001 * (y - y_hat)*parameters['W2'][1][0]*A1[1][0]*(1-A1[1][0])*X[0][0])
  parameters['W1'][1][1] = parameters['W1'][1][1] + (0.0001 * (y - y_hat)*parameters['W2'][1][0]*A1[1][0]*(1-A1[1][0])*X[1][0])
  parameters['b1'][1][0] = parameters['b1'][1][0] + (0.0001 * (y - y_hat)*parameters['W2'][1][0]*A1[1][0]*(1-A1[1][0]))

### For 1st Student

In [23]:
# Extract the features 'cgpa' and 'profile_score' from the first row of the DataFrame
# and reshape it into a column vector of shape (2, 1), where 2 is the number of features
# and 1 is the number of training examples.
X = df[['cgpa', 'profile_score']].values[0].reshape(2,1)

# Extract the target label 'placed' for the first row from the DataFrame.
y = df[['placed']].values[0][0]

# Initialize the parameters for a neural network with 2 input features, 2 neurons in the hidden layer, and 1 output neuron.
parameters = initialize_parameters([2, 2, 1])

# Perform forward propagation to compute the predicted value (y_hat) and the activation from the first layer (A1).
y_hat, A1 = L_layer_forward(X, parameters)

# Extract the scalar value from the predicted output (y_hat) for easier use in calculations.
y_hat = y_hat[0][0]

# Update the parameters of the neural network based on the prediction error, using backpropagation.
update_parameters(parameters, y, y_hat, A1, X)

# Compute and print the loss (binary cross-entropy loss) for this specific training example.
print('Loss for this student - ', -y*np.log(y_hat) - (1-y)*np.log(1-y_hat))

# Display the updated parameters after the backpropagation step.
parameters


Loss for this student -  0.613402628898913


{'W1': array([[0.10000513, 0.10000513],
        [0.10000513, 0.10000513]]),
 'b1': array([[6.41054186e-07],
        [6.41054186e-07]]),
 'W2': array([[0.10003815, 0.1       ],
        [0.10003815, 0.1       ]]),
 'b2': array([[0.100084]])}

### For 2nd Student

In [24]:
# Extract the features 'cgpa' and 'profile_score' from the second row of the DataFrame
# and reshape it into a column vector of shape (2, 1), where 2 is the number of features
# and 1 is the number of training examples.
X = df[['cgpa', 'profile_score']].values[1].reshape(2,1)

# Extract the target label 'placed' for the second row from the DataFrame.
y = df[['placed']].values[1][0]

# Perform forward propagation to compute the predicted value (y_hat) and the activation from the first layer (A1).
y_hat, A1 = L_layer_forward(X, parameters)

# Extract the scalar value from the predicted output (y_hat) for easier use in calculations.
y_hat = y_hat[0][0]

# Update the parameters of the neural network based on the prediction error, using backpropagation.
update_parameters(parameters, y, y_hat, A1, X)

# Compute and print the loss (binary cross-entropy loss) for this specific training example.
print('Loss for this student - ', -y * np.log(y_hat) - (1 - y) * np.log(1 - y_hat))

# Display the updated parameters after the backpropagation step.
parameters


Loss for this student -  0.568725622654268


{'W1': array([[0.10000937, 0.10001059],
        [0.10000937, 0.10001059]]),
 'b1': array([[1.24770113e-06],
        [1.24770113e-06]]),
 'W2': array([[0.10007424, 0.1       ],
        [0.10007424, 0.1       ]]),
 'b2': array([[0.10011761]])}

### For 3rd Student

In [25]:
# Extract the features 'cgpa' and 'profile_score' from the third row of the DataFrame
# and reshape it into a column vector of shape (2, 1), where 2 is the number of features
# and 1 is the number of training examples.
X = df[['cgpa', 'profile_score']].values[2].reshape(2,1)

# Extract the target label 'placed' for the third row from the DataFrame.
y = df[['placed']].values[2][0]

# Perform forward propagation to compute the predicted value (y_hat) and the activation from the first layer (A1).
y_hat, A1 = L_layer_forward(X, parameters)

# Extract the scalar value from the predicted output (y_hat) for easier use in calculations.
y_hat = y_hat[0][0]

# Update the parameters of the neural network based on the prediction error, using backpropagation.
update_parameters(parameters, y, y_hat, A1, X)

# Compute and print the loss (binary cross-entropy loss) for this specific training example.
print('Loss for this student - ', -y * np.log(y_hat) - (1 - y) * np.log(1 - y_hat))

# Display the updated parameters after the backpropagation step.
parameters


Loss for this student -  0.8353333695154365


{'W1': array([[0.10000463, 0.10000267],
        [0.10000463, 0.10000267]]),
 'b1': array([[4.56125378e-07],
        [4.56135584e-07]]),
 'W2': array([[0.10002712, 0.1       ],
        [0.10002712, 0.1       ]]),
 'b2': array([[0.09997049]])}

### For 4th Student

In [21]:
X = df[['cgpa', 'profile_score']].values[3].reshape(2,1) # Shape(no of features, no. of training example)
y = df[['placed']].values[3][0]

y_hat,A1 = L_layer_forward(X,parameters)
y_hat = y_hat[0][0]

update_parameters(parameters,y,y_hat,A1,X)

print('Loss for this student - ',-y*np.log(y_hat) - (1-y)*np.log(1-y_hat))

parameters

Loss for this student -  0.8239406691217557


{'W1': array([[0.10000386, 0.10000507],
        [0.10000386, 0.10000507]]),
 'b1': array([[1.43799372e-07],
        [1.43805564e-07]]),
 'W2': array([[0.1000332, 0.1      ],
        [0.1000332, 0.1      ]]),
 'b2': array([[0.09997707]])}

### epochs implementation

In [22]:
parameters = initialize_parameters([2,2,1])
epochs = 50

for i in range(epochs):

  Loss = []

  for j in range(df.shape[0]):

    X = df[['cgpa', 'profile_score']].values[j].reshape(2,1) # Shape(no of features, no. of training example)
    y = df[['placed']].values[j][0]

    # Parameter initialization


    y_hat,A1 = L_layer_forward(X,parameters)
    y_hat = y_hat[0][0]

    update_parameters(parameters,y,y_hat,A1,X)

    Loss.append(-y*np.log(y_hat) - (1-y)*np.log(1-y_hat))

  print('Epoch - ',i+1,'Loss - ',np.array(Loss).mean())

parameters

Epoch -  1 Loss -  0.7103199085929446
Epoch -  2 Loss -  0.6991702892802629
Epoch -  3 Loss -  0.6991679314811485
Epoch -  4 Loss -  0.6991655746710999
Epoch -  5 Loss -  0.6991632188496667
Epoch -  6 Loss -  0.699160864016399
Epoch -  7 Loss -  0.6991585101708473
Epoch -  8 Loss -  0.6991561573125619
Epoch -  9 Loss -  0.6991538054410936
Epoch -  10 Loss -  0.6991514545559935
Epoch -  11 Loss -  0.6991491046568126
Epoch -  12 Loss -  0.6991467557431024
Epoch -  13 Loss -  0.6991444078144144
Epoch -  14 Loss -  0.6991420608703007
Epoch -  15 Loss -  0.6991397149103132
Epoch -  16 Loss -  0.6991373699340042
Epoch -  17 Loss -  0.6991350259409265
Epoch -  18 Loss -  0.6991326829306324
Epoch -  19 Loss -  0.6991303409026751
Epoch -  20 Loss -  0.699127999856608
Epoch -  21 Loss -  0.6991256597919842
Epoch -  22 Loss -  0.6991233207083575
Epoch -  23 Loss -  0.6991209826052818
Epoch -  24 Loss -  0.699118645482311
Epoch -  25 Loss -  0.6991163093389996
Epoch -  26 Loss -  0.699113974174902

{'W1': array([[0.09994267, 0.09984548],
        [0.09994272, 0.09984548]]),
 'b1': array([[-3.38405750e-05],
        [-3.38419977e-05]]),
 'W2': array([[0.09920806, 0.1       ],
        [0.09920816, 0.1       ]]),
 'b2': array([[0.09915209]])}